In [1]:
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder \
    .appName("Read CSV GZ Example") \
    .getOrCreate()

24/10/01 22:20:51 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
listings = spark.read.csv("data/listings.csv.gz", 
    header=True,
    inferSchema=True,
    sep=",",        # The separator is still a comma
    quote='"',      # Quote character
    escape='"',    # No escape character
    multiLine=True, # Allows for multiline records
    mode="PERMISSIVE"  # Handle malformed lines permissively
)

In [3]:
review_locations = listings.select(listings.review_scores_location)
review_locations.show()

+----------------------+
|review_scores_location|
+----------------------+
|                  4.72|
|                  4.93|
|                  4.89|
|                  4.59|
|                  4.67|
|                  4.66|
|                  4.54|
|                  4.83|
|                  NULL|
|                  4.72|
|                  4.92|
|                  4.88|
|                  4.83|
|                  4.89|
|                  4.77|
|                  4.79|
|                   5.0|
|                  4.76|
|                   4.5|
|                  4.64|
+----------------------+
only showing top 20 rows



In [5]:
listings \
  .select(listings.review_scores_location) \
  .show()

+----------------------+
|review_scores_location|
+----------------------+
|                  4.72|
|                  4.93|
|                  4.89|
|                  4.59|
|                  4.67|
|                  4.66|
|                  4.54|
|                  4.83|
|                  NULL|
|                  4.72|
|                  4.92|
|                  4.88|
|                  4.83|
|                  4.89|
|                  4.77|
|                  4.79|
|                   5.0|
|                  4.76|
|                   4.5|
|                  4.64|
+----------------------+
only showing top 20 rows



In [8]:
high_score_listings = listings \
  .filter(listings.review_scores_location > 4.5) \
  .select('id', 'price', 'name', 'review_scores_location')

high_score_listings.show(20, truncate=False)


+------+-------+--------------------------------------------------+----------------------+
|id    |price  |name                                              |review_scores_location|
+------+-------+--------------------------------------------------+----------------------+
|13913 |$59.00 |Holiday London DB Room Let-on going               |4.72                  |
|15400 |$120.00|Bright Chelsea  Apartment. Chelsea!               |4.93                  |
|17402 |$493.00|Fab 3-Bed/2 Bath & Wifi: Trendy W1                |4.89                  |
|24328 |$190.00|Battersea live/work artist house, garden & parking|4.59                  |
|33332 |$140.00|Beautiful Ensuite Richmond-upon-Thames borough    |4.67                  |
|116268|$40.00 |Double Room (Unavailable for check in 31Dec-1Jan) |4.66                  |
|117203|$131.00|A stylish Victorian home in West London           |4.54                  |
|127652|$215.00|Contemporary central London apt                   |4.83                  |

In [10]:
high_score_listings.dropna().show(20, truncate=False)

+------+-------+--------------------------------------------------+----------------------+
|id    |price  |name                                              |review_scores_location|
+------+-------+--------------------------------------------------+----------------------+
|13913 |$59.00 |Holiday London DB Room Let-on going               |4.72                  |
|15400 |$120.00|Bright Chelsea  Apartment. Chelsea!               |4.93                  |
|17402 |$493.00|Fab 3-Bed/2 Bath & Wifi: Trendy W1                |4.89                  |
|24328 |$190.00|Battersea live/work artist house, garden & parking|4.59                  |
|33332 |$140.00|Beautiful Ensuite Richmond-upon-Thames borough    |4.67                  |
|116268|$40.00 |Double Room (Unavailable for check in 31Dec-1Jan) |4.66                  |
|117203|$131.00|A stylish Victorian home in West London           |4.54                  |
|127652|$215.00|Contemporary central London apt                   |4.83                  |

In [15]:
high_score_listings.schema['price']

StructField('price', StringType(), True)

In [16]:
from pyspark.sql.functions import regexp_replace

price_num_df = listings \
  .withColumn('price_num', regexp_replace('price', '[$,]', '').cast('float')) \

price_num_df.schema['price_num']

StructField('price_num', FloatType(), True)

In [18]:
price_num_df \
  .select('price_num', 'name') \
  .show(20, truncate=False)

+---------+--------------------------------------------------+
|price_num|name                                              |
+---------+--------------------------------------------------+
|59.0     |Holiday London DB Room Let-on going               |
|120.0    |Bright Chelsea  Apartment. Chelsea!               |
|493.0    |Fab 3-Bed/2 Bath & Wifi: Trendy W1                |
|190.0    |Battersea live/work artist house, garden & parking|
|140.0    |Beautiful Ensuite Richmond-upon-Thames borough    |
|40.0     |Double Room (Unavailable for check in 31Dec-1Jan) |
|131.0    |A stylish Victorian home in West London           |
|215.0    |Contemporary central London apt                   |
|NULL     |Double bedroom in cottage Twickenham (sleeps 1-2) |
|150.0    |Apartment close to centre.                        |
|145.0    |I Bedroom flat Tower of London                    |
|NULL     |Elegant Very Central: Fitzrovia W1. King Bed, Wifi|
|109.0    |Bright 1 bedroom off brick land             

In [23]:
price_num_df.filter( (price_num_df.price_num < 100) & (price_num_df.review_scores_location > 4.5)) \
  .select('name', 'price', 'review_scores_location') \
  .show(truncate=False)

+--------------------------------------------------+------+----------------------+
|name                                              |price |review_scores_location|
+--------------------------------------------------+------+----------------------+
|Holiday London DB Room Let-on going               |$59.00|4.72                  |
|Double Room (Unavailable for check in 31Dec-1Jan) |$40.00|4.66                  |
|You are GUARANTEED to love this                   |$82.00|4.77                  |
|Room 1 Large Double Bedroom - front ground floor  |$55.00|5.0                   |
|SPACIOUS ROOM IN CONTEMPORARY STYLE FLAT          |$55.00|4.76                  |
|West London-W7, Hanwell(Area Ealing) Room (Female)|$37.00|4.64                  |
|West London,loft ensuite, 5min2tube               |$69.00|4.75                  |
|Charming Flat in Notting Hill                     |$94.00|4.87                  |
|Room with a view, shared flat,  central  Bankside |$86.00|4.86                  |
|You

In [25]:
price_num_df.filter('price_num < 100 AND review_scores_location > 4.5') \
  .select('name', 'price', 'review_scores_location') \
  .show(truncate=False)

+--------------------------------------------------+------+----------------------+
|name                                              |price |review_scores_location|
+--------------------------------------------------+------+----------------------+
|Holiday London DB Room Let-on going               |$59.00|4.72                  |
|Double Room (Unavailable for check in 31Dec-1Jan) |$40.00|4.66                  |
|You are GUARANTEED to love this                   |$82.00|4.77                  |
|Room 1 Large Double Bedroom - front ground floor  |$55.00|5.0                   |
|SPACIOUS ROOM IN CONTEMPORARY STYLE FLAT          |$55.00|4.76                  |
|West London-W7, Hanwell(Area Ealing) Room (Female)|$37.00|4.64                  |
|West London,loft ensuite, 5min2tube               |$69.00|4.75                  |
|Charming Flat in Notting Hill                     |$94.00|4.87                  |
|Room with a view, shared flat,  central  Bankside |$86.00|4.86                  |
|You

In [30]:
listings \
  .select(listings.property_type) \
  .distinct() \
  .show(truncate=False)

+----------------------------------+
|property_type                     |
+----------------------------------+
|Private room in lighthouse        |
|Private room in loft              |
|Private room in earthen home      |
|Entire chalet                     |
|Earthen home                      |
|Shared room in bus                |
|Farm stay                         |
|Entire rental unit                |
|Shared room in hostel             |
|Shared room                       |
|Private room in condo             |
|Room in boutique hotel            |
|Private room in religious building|
|Room in bed and breakfast         |
|Private room in casa particular   |
|Entire guesthouse                 |
|Private room in bungalow          |
|Entire cabin                      |
|Hut                               |
|Private room in nature lodge      |
+----------------------------------+
only showing top 20 rows



In [31]:
listings \
  .select(listings.property_type, listings.room_type) \
  .distinct() \
  .show(truncate=False)

+----------------------------------+---------------+
|property_type                     |room_type      |
+----------------------------------+---------------+
|Private room in casa particular   |Private room   |
|Room in hostel                    |Hotel room     |
|Dome                              |Entire home/apt|
|Entire serviced apartment         |Entire home/apt|
|Private room in loft              |Private room   |
|Shipping container                |Entire home/apt|
|Private room in villa             |Private room   |
|Farm stay                         |Entire home/apt|
|Room in hotel                     |Hotel room     |
|Shared room in rental unit        |Shared room    |
|Private room in guest suite       |Private room   |
|Room in rental unit               |Hotel room     |
|Room in serviced apartment        |Hotel room     |
|Private room in serviced apartment|Private room   |
|Private room in hostel            |Private room   |
|Shared room                       |Shared roo

In [34]:
listings \
  .select(listings.property_type) \
  .distinct() \
  .write \
  .csv('data/property_types')